### Scraping to txt files

In [ ]:
# imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
import io

news_contents = []
n=1
df=()
df3 = pd.read_csv('urls_to_scrap.csv')
df3 = df3.sample(frac=1).reset_index(drop=True).dropna()

options = webdriver.ChromeOptions()
options.add_argument('headless')
m=len(df3)

for url in df3['URL']:
    print("Converting " + url +"  URL Number ---> " + str(n)+" (Out of ---> " + str(m)+")")
    n=n+1
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}
    driver = webdriver.Chrome(executable_path='chromedriver',chrome_options=options)
    driver.get(url)
    soup1 = BeautifulSoup(driver.page_source, 'html5lib')
    #coverpage_news = soup1.find_all("div", class_="text")
    coverpage_news = soup1.find_all("p")
    
    list_paragraphs = []
    final_article = []
    for p in np.arange(0, len(coverpage_news)):
        paragraph = coverpage_news[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
    fname = str(n)+'Values.txt'
    with io.open(fname, "w", encoding="utf-8") as f:
        f.write(final_article)

### Scraping to df

In [ ]:
# imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
news_contents = []
n=1
df=()
df3 = pd.read_csv('urls_to_scrap.csv')
df3 = df3.sample(frac=1).reset_index(drop=True).dropna()

options = webdriver.ChromeOptions()
options.add_argument('headless')
m=len(df3)

for url in df3['URL']:
    print("Converting " + url +"  URL Number ---> " + str(n)+" (Out of ---> " + str(m)+")")
    n=n+1
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}
    driver = webdriver.Chrome(executable_path='chromedriver',chrome_options=options)
    driver.get(url)
    soup1 = BeautifulSoup(driver.page_source, 'html5lib')
    #coverpage_news = soup1.find_all("div", class_="text")
    coverpage_news = soup1.find_all("p")
    
    list_paragraphs = []
    for p in np.arange(0, len(coverpage_news)):
        paragraph = coverpage_news[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
    news_contents.append(final_article)
    df = pd.DataFrame(news_contents)
df
header=['content']
df.to_csv('content_for_analysis1.csv', columns = header)


In [ ]:
pd.set_option('max_colwidth', 80000)
df

# scraping trip advisor reviews

### Scrape a destination page to get all reviews pages

In [ ]:
from IPython.display import clear_output
# imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
import time

url="https://www.tripadvisor.com/Attractions-g187147-Activities-Paris_Ile_de_France.html"
profile = webdriver.FirefoxProfile()
# We set the coordinate of where we want to be.
profile.set_preference("geo.wifi.uri", 'data:application/json,{"location": {"lat": 38.912650, "lng":-77.036185}, "accuracy": 20.0}')
# This line is necessary to avoid to prompt for geolocation authorization.
profile.set_preference("geo.prompt.testing", True)
driver = webdriver.Firefox(profile)
driver.get(url)
time.sleep(5)
list_paragraphs=[]
number_of_pages = int(driver.find_elements_by_xpath("//a[contains(@class,'pageNum')]")[5].text)
for i in range(number_of_pages - 1):
    time.sleep(2)
    for ii in driver.find_elements_by_xpath('//a[contains(@href, "Review")]'):
        paragraph = ii.get_attribute('href')
        list_paragraphs.append(paragraph)
    clear_output()
    print(list_paragraphs)
    element = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, '//a[contains(@class ,"nav")]')))
    driver.find_element_by_partial_link_text('Next').click()


In [ ]:
list_paragraphs=pd.DataFrame(list_paragraphs)
list_paragraphs=list_paragraphs.rename(columns={0: "URL",})
list_paragraphs=pd.DataFrame(list_paragraphs)
#list_paragraphs=list_paragraphs.rename(columns={"content": "URL",})
list_paragraphs=list_paragraphs.URL.str.replace("#REVIEWS","")
list_paragraphs=pd.DataFrame(list_paragraphs)
list_paragraphs.head()

In [ ]:
df=list_paragraphs
df=df[~df['URL'].str.contains('VacationRentals')]
df=df.drop_duplicates()
df

In [ ]:
df3 = pd.read_csv('urls_to_scrap2.csv')

df=pd.concat([list_paragraphs,df3])

In [ ]:
import os
import csv
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from IPython.display import clear_output


profile = webdriver.FirefoxProfile()
# We set the coordinate of where we want to be.
profile.set_preference("geo.wifi.uri", 'data:application/json,{"location": {"lat": 38.912650, "lng":-77.036185}, "accuracy": 20.0}')
# This line is necessary to avoid to prompt for geolocation authorization.
profile.set_preference("geo.prompt.testing", True)
driver = webdriver.Firefox(profile)


# function to check if the button is on the page, to avoid miss-click problem
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

#WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//span[contains(@class, 'list-parts-ExpandableReview')]"))).click()
#driver.find_element_by_xpath("//span[contains(@class, 'list-parts-ExpandableReview')]").click()
df3 = df
df_Trip_final=pd.DataFrame()
a=0
b=0
news_contents_Trip = []
for url in df['URL']:
    driver.get(url)
    time.sleep(5)
    # change the value inside the range to save more or less reviews
    try:
        number_of_pages = int(driver.find_elements_by_xpath("//a[contains(@class,'pageNum')]")[5].text)
        number_of_pages = int(number_of_pages/10)
    except:
        number_of_pages=2
        continue
    for i in range(number_of_pages - 1):
        time.sleep(5)
        count=a+b
        print("done"+"  "+str(i)+" "+"pages out of:  "+ str(number_of_pages))
        print("============================")
        print("===========================")
        print('error count:   '+str(count))
        list_paragraphs = []
        if (check_exists_by_xpath("//div[contains(@class, 'list-parts-ExpandableReview')]")):
            # to expand the review
            try:
                element = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, '//span[contains(@class ,"_36B4Vw6t")]')))
                element.click();
                time.sleep(0.5)
                ids=driver.find_elements_by_xpath('//q[contains(@class ,"review-list-parts")]')
            except:
                print("error 1")
                a=a+1
                pass
            for ii in range(len(ids)):
                try:
                    element = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, '//q[contains(@class ,"review-list-parts")]')))
                    time.sleep(0.5)
                    paragraph = ids[ii].text
                    print("done"+"  "+str(ii)+" "+"reviews")
                    list_paragraphs.append(paragraph)
                    news_contents_Trip=pd.DataFrame(list_paragraphs)
                    #final_article_Trip = " ".join(list_paragraphs)
                except:
                    print("error 2")
                    b=b+1
                    pass
            # to change the page
            time.sleep(0.5)
            clear_output()
            driver.find_element_by_partial_link_text('Next').click()
        df_Trip_final = pd.concat([df_Trip_final,news_contents_Trip])
        clear_output()
        print(df_Trip_final.tail())
        
df_Trip_final.to_csv('content_for_analysis_trip.csv')
driver.close()

In [ ]:
df_Trip_final.to_csv('content_for_analysis.csv')
